In [1]:
import boto3
from pathlib import Path

In [2]:
TRAINING_BUCKET = 'markpett-training'

In [3]:
s3 = boto3.resource('s3')
bucket = s3.Bucket(TRAINING_BUCKET)
custom_files = list(bucket.objects.filter(Prefix='custom_files/'))
custom_files

[s3.ObjectSummary(bucket_name='markpett-training', key='custom_files/hyperparameters.json'),
 s3.ObjectSummary(bucket_name='markpett-training', key='custom_files/model_metadata.json'),
 s3.ObjectSummary(bucket_name='markpett-training', key='custom_files/reward_function.py')]

In [ ]:
# check size of everything
total = sum(a.size for a in bucket.objects.all())
print(f'{total / 1024 / 1024:,} MB')

In [10]:
# download all files
for obj in custom_files:
    save_name = Path(obj.key).name
    bucket.download_file(obj.key, save_name)
    print(save_name)

hyperparameters.json
model_metadata.json
reward_function.py


In [11]:
# download env files
env_files = list(bucket.objects.filter(Prefix='env_files/'))
for obj in env_files:
    save_name = Path(obj.key).name
    bucket.download_file(obj.key, save_name)
    print(save_name)

run.env
system.env


In [12]:
# download startup_files
env_files = list(bucket.objects.filter(Prefix='startup_files/'))
for obj in env_files:
    save_name = Path(obj.key).name
    bucket.download_file(obj.key, save_name)
    print(save_name)

begin_dr.sh
can_overwrite_folder.py
is_bucket_changing.py


In [7]:
fname = 'hyperparameters.json'
bucket.upload_file(fname, f'custom_files/{fname}')
print(fname)

hyperparameters.json


In [ ]:
fname = 'model_metadata.json'
bucket.upload_file(fname, f'custom_files/{fname}')
print(fname)

In [8]:
fname = 'reward_function.py'
bucket.upload_file(fname, f'custom_files/{fname}')
print(fname)

reward_function.py


In [9]:
fname = 'run.env'
bucket.upload_file(fname, f'env_files/{fname}')
print(fname)

run.env


In [ ]:
fname = 'system.env'
bucket.upload_file(fname, f'env_files/{fname}')
print(fname)

In [ ]:
fname = 'begin_dr.sh'
bucket.upload_file(fname, f'startup_files/{fname}')
print(fname)

In [ ]:
fname = 'is_bucket_changing.py'
bucket.upload_file(fname, f'startup_files/{fname}')
print(fname)

In [ ]:
fname = 'can_overwrite_folder.py'
bucket.upload_file(fname, f'startup_files/{fname}')
print(fname)

In [ ]:
import argparse
import boto3
import os
import subprocess
from time import sleep
from traceback import print_exc
from typing import cast, NoReturn

DR_LOCAL_S3_BUCKET = 'markpett-training'
DR_LOCAL_S3_MODEL_PREFIX = '07-dummy-test'
DR_LOCAL_S3_METRICS_PREFIX = '07-dummy-test/metrics'
# DR_LOCAL_S3_BUCKET = os.getenv('DR_LOCAL_S3_BUCKET')
# DR_LOCAL_S3_METRICS_PREFIX = os.getenv('DR_LOCAL_S3_METRICS_PREFIX')


def get_interval() -> float:
    parser = argparse.ArgumentParser(
        description='keep checking s3 bucket at given interval')
    parser.add_argument(
        'interval_secs', type=float,
        help='interval seconds as float')
    args = parser.parse_args()
    return cast(float, args.interval_secs)


def keep_checking_metrics(interval_secs: float) -> NoReturn:
    prev_size = 0
    obj_name = None
    while True:
        sleep(interval_secs)
        try:
            if DR_LOCAL_S3_BUCKET is None \
                    or DR_LOCAL_S3_METRICS_PREFIX is None:
                print('Missing environment vars: DR_LOCAL_S3_BUCKET'
                      + ', DR_LOCAL_S3_METRICS_PREFIX')
                reboot_system()
            else:
                obj_name = f'{DR_LOCAL_S3_METRICS_PREFIX}/TrainingMetrics.json'
                size = get_obj_size(obj_name)
                if size == prev_size:
                    print('No change in size of object: '
                          + f'S3://{DR_LOCAL_S3_BUCKET}/{obj_name}')
                    print(f'Elapsed {interval_secs} secs: '
                          + f'prev_size={prev_size}, size_now={size}')
                    reboot_system()
                prev_size = size
        except Exception:
            print_exc()
            reboot_system()


def get_obj_size(key: str) -> int:
    s3 = boto3.resource('s3')
    bucket = s3.Bucket(DR_LOCAL_S3_BUCKET)
    obj = bucket.Object(key)
    return obj.content_length


def reboot_system() -> None:
    print('Rebooting')
    # subprocess.run(['sudo', 'shutdown', '-r', '1'])


def main() -> None:
    # interval_secs = get_interval()
    interval_secs = 1.0
    print(f'checking s3 bucket at {interval_secs} second intervals')
    keep_checking_metrics(interval_secs)


if __name__ == '__main__':
    main()


In [ ]:
import boto3
import os

DR_LOCAL_S3_BUCKET = 'markpett-training'
DR_LOCAL_S3_MODEL_PREFIX = '10c-SquaredRoss-1'
DR_LOCAL_S3_METRICS_PREFIX = '10c-SquaredRoss-1/metrics'

# DR_LOCAL_S3_BUCKET = os.getenv('DR_LOCAL_S3_BUCKET')
# DR_LOCAL_S3_MODEL_PREFIX = os.getenv('DR_LOCAL_S3_MODEL_PREFIX')
# DR_LOCAL_S3_METRICS_PREFIX = os.getenv('DR_LOCAL_S3_METRICS_PREFIX')


def obj_exists(bucket: str, prefix: str, key: str) -> bool:
    s3 = boto3.resource('s3')
    resource = s3.Bucket(bucket)
    objs = list(resource.objects.filter(Prefix=prefix))
    return key in [x.key for x in objs]


def is_metric_present() -> bool:
    return obj_exists(
        DR_LOCAL_S3_BUCKET,
        DR_LOCAL_S3_MODEL_PREFIX,
        f'{DR_LOCAL_S3_METRICS_PREFIX}/TrainingMetrics.json'
    )


def main() -> None:
    '''
    Prints "overwrite" to the console if the folder is safe to overwrite.
    Notionally, this is if not any exceptions were thrown when reading
    the folder, and that the metrics file is not present in the folder.
    '''
    if not is_metric_present():
        print('overwrite', end='')


if __name__ == '__main__':
    main()


In [ ]:
##########

# run script as different user
runuser --login ubuntu << 'EOF'

cd /home/ubuntu
source .profile

# Installing packages
sudo apt install --no-install-recommends -y jq awscli python3-boto3

# include net.ipv4.vs.xxxxxx points
sudo bash -c 'echo ip_vs >> /etc/modules-load.d/modules.conf'

# Installing Docker
sudo apt install -y --no-install-recommends docker.io docker-buildx docker-compose-v2
sudo systemctl enable docker
sudo systemctl restart docker

# allow docker access
sudo usermod -a -G docker ubuntu

EOF


# run script as different user
runuser --login ubuntu << 'EOF'

cd /home/ubuntu
source .profile

git clone -b {{ versionTag }} https://github.com/aws-deepracer-community/deepracer-for-cloud.git

cd deepracer-for-cloud/bin
./init.sh -c aws -a cpu -s compose

cd /home/ubuntu

cat << 'EOFB' > do_startup.sh
#!/bin/bash
cd /home/ubuntu
source .profile
aws s3 cp s3://markpett-training/startup_files/begin_dr.sh begin_dr.sh
bash begin_dr.sh
EOFB

chmod u+x do_startup.sh

cat << 'EOFB' > my_crontab
SHELL=/bin/bash
@reboot /home/ubuntu/do_startup.sh
EOFB

crontab -u ubuntu my_crontab

EOF
